In [25]:
import string

ALPHABET = string.ascii_uppercase

def clean_text(text):
    return ''.join(ch for ch in text.upper() if ch.isalpha())

def extend_key(key, length):
    key = clean_text(key)
    return (key * (length // len(key) + 1))[:length]

def vigenere_encrypt(plaintext, key):
    pt = clean_text(plaintext)
    k  = extend_key(key, len(pt))
    ciphertext = ""
    modulo_rows = []

    for i, (p, kk) in enumerate(zip(pt, k)):
        p_idx = ALPHABET.index(p)
        k_idx = ALPHABET.index(kk)
        c_idx = (p_idx + k_idx) % 26
        c = ALPHABET[c_idx]
        ciphertext += c

        modulo_rows.append({
            "index": i,
            "P": p,
            "P_idx": p_idx,
            "K": kk,
            "K_idx": k_idx,
            "(P_idx + K_idx) % 26": c_idx,
            "C": c
        })
    return ciphertext, modulo_rows, pt, k

def vigenere_decrypt(ciphertext, key):
    ct = clean_text(ciphertext)
    k  = extend_key(key, len(ct))
    plaintext = ""
    for c, kk in zip(ct, k):
        c_idx = ALPHABET.index(c)
        k_idx = ALPHABET.index(kk)
        p_idx = (c_idx - k_idx) % 26
        p = ALPHABET[p_idx]
        plaintext += p
    return plaintext

def print_vigenere_demo(plaintext, key):
    ciphertext, table, pt_clean, key_full = vigenere_encrypt(plaintext, key)
    decrypted = vigenere_decrypt(ciphertext, key)

    print("=== VIGENERE CIPHER ===")
    print(f"Plaintext  : {pt_clean}")
    print(f"Key        : {key_full}")
    print(f"Ciphertext : {ciphertext}")
    print(f"Dekripsi   : {decrypted}")
    print()
    print("Tabel perhitungan modulo (per karakter):")
    print("{:>3} {:>3} {:>5} {:>3} {:>5} {:>12} {:>3}".format(
        "i", "P", "P_idx", "K", "K_idx", "(P+K)%26", "C"
    ))
    for row in table:
        print("{:>3} {:>3} {:>5} {:>3} {:>5} {:>12} {:>3}".format(
            row["index"],
            row["P"],
            row["P_idx"],
            row["K"],
            row["K_idx"],
            row["(P_idx + K_idx) % 26"],
            row["C"]
        ))

plaintext = "PERUNGGU"
key       = "INI ABADI"

print_vigenere_demo(plaintext, key)


=== VIGENERE CIPHER ===
Plaintext  : PERUNGGU
Key        : INIABADI
Ciphertext : XRZUOGJC
Dekripsi   : PERUNGGU

Tabel perhitungan modulo (per karakter):
  i   P P_idx   K K_idx     (P+K)%26   C
  0   P    15   I     8           23   X
  1   E     4   N    13           17   R
  2   R    17   I     8           25   Z
  3   U    20   A     0           20   U
  4   N    13   B     1           14   O
  5   G     6   A     0            6   G
  6   G     6   D     3            9   J
  7   U    20   I     8            2   C


In [27]:
import string

def prepare_key_playfair(key):
    key = ''.join(ch for ch in key.upper() if ch.isalpha())
    key = key.replace('J', 'I')
    seen = set()
    result = []
    for ch in key:
        if ch not in seen:
            seen.add(ch)
            result.append(ch)
    for ch in string.ascii_uppercase:
        if ch == 'J':
            continue
        if ch not in seen:
            seen.add(ch)
            result.append(ch)

    matrix = [result[i*5:(i+1)*5] for i in range(5)]
    return matrix

def find_position(matrix, ch):
    if ch == 'J':
        ch = 'I'
    for r in range(5):
        for c in range(5):
            if matrix[r][c] == ch:
                return r, c
    raise ValueError("Character not found in matrix")

def prepare_plaintext_playfair(plaintext):
    text = ''.join(ch for ch in plaintext.upper() if ch.isalpha())
    text = text.replace('J', 'I')
    pairs = []
    i = 0
    while i < len(text):
        a = text[i]
        b = ''
        if i + 1 < len(text):
            b = text[i+1]
        if a == b:
            b = 'X'
            i += 1
        else:
            i += 2
        pairs.append(a + (b if b else 'X'))
    return pairs

def playfair_encrypt_pair(a, b, matrix):
    r1, c1 = find_position(matrix, a)
    r2, c2 = find_position(matrix, b)
    if r1 == r2:
        return matrix[r1][(c1+1) % 5] + matrix[r2][(c2+1) % 5]
    elif c1 == c2:
        return matrix[(r1+1) % 5][c1] + matrix[(r2+1) % 5][c2]
    else:
        return matrix[r1][c2] + matrix[r2][c1]

def playfair_decrypt_pair(a, b, matrix):
    r1, c1 = find_position(matrix, a)
    r2, c2 = find_position(matrix, b)
    if r1 == r2:
        return matrix[r1][(c1-1) % 5] + matrix[r2][(c2-1) % 5]
    elif c1 == c2:
        return matrix[(r1-1) % 5][c1] + matrix[(r2-1) % 5][c2]
    else:
        return matrix[r1][c2] + matrix[r2][c1]

def playfair_encrypt(plaintext, key):
    matrix = prepare_key_playfair(key)
    pairs = prepare_plaintext_playfair(plaintext)
    ciphertext = ""
    for a, b in pairs:
        ciphertext += playfair_encrypt_pair(a, b, matrix)
    return ciphertext, matrix, pairs

def playfair_decrypt(ciphertext, key):
    matrix = prepare_key_playfair(key)
    pairs = [ciphertext[i:i+2] for i in range(0, len(ciphertext), 2)]
    plaintext = ""
    for a, b in pairs:
        plaintext += playfair_decrypt_pair(a, b, matrix)
    return plaintext, matrix, pairs

def print_matrix(matrix):
    for row in matrix:
        print(' '.join(row))

def print_playfair_demo(plaintext, key):
    ciphertext, matrix, pairs = playfair_encrypt(plaintext, key)
    decrypted, _, _ = playfair_decrypt(ciphertext, key)

    print("=== PLAYFAIR CIPHER ===")
    print(f"Key       : {key.upper()}")
    print("Matriks 5x5:")
    print_matrix(matrix)
    print()
    print(f"Plaintext : {plaintext}")
    print(f"Digraf    : {pairs}")
    print(f"Ciphertext: {ciphertext}")
    print(f"Dekripsi  : {decrypted}")

plaintext_pf = "PERUNGGU"
key_pf       = "DALAM DINAMIKA"

print_playfair_demo(plaintext_pf, key_pf)


=== PLAYFAIR CIPHER ===
Key       : DALAM DINAMIKA
Matriks 5x5:
D A L M I
N K B C E
F G H O P
Q R S T U
V W X Y Z

Plaintext : PERUNGGU
Digraf    : ['PE', 'RU', 'NG', 'GU']
Ciphertext: UPSQKFPR
Dekripsi  : PERUNGGU
